<a href="https://colab.research.google.com/github/Suryansh1089/GroupProject/blob/main/SarcasmDetectionBaseline2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [38]:
twitter_train=pd.read_json('twitter_train.jsonl',lines=True)

In [39]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [79]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')

In [80]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [81]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')

In [82]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [83]:
twitter_train.tail()

,label,response,context
4995,NOT_SARCASM,you dont i have purchased a lot on amazon c...,apologies for the inconvenience you faced wit...
4996,NOT_SARCASM,emotions you say never knew that i think i ...,idk tho i think i m hungry but that s de...
4997,NOT_SARCASM,you are so right yes silence is not priv...,peace to you and two countries certainly s...
4998,NOT_SARCASM,another lazy delusional voter who takes the...,bernie sanders told elizabeth warren in privat...
4999,NOT_SARCASM,i hope you know no news outlet from nigeria ...,pdp protest brainstorming session deji we nee...


In [84]:
from collections import Counter

In [85]:
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [86]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [87]:
train_label=le.fit_transform(twitter_train['label'])

In [88]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [89]:
twitter_train.tail()

,label,response,context
4995,NOT_SARCASM,you dont i have purchased a lot on amazon c...,apologies for the inconvenience you faced wit...
4996,NOT_SARCASM,emotions you say never knew that i think i ...,idk tho i think i m hungry but that s de...
4997,NOT_SARCASM,you are so right yes silence is not priv...,peace to you and two countries certainly s...
4998,NOT_SARCASM,another lazy delusional voter who takes the...,bernie sanders told elizabeth warren in privat...
4999,NOT_SARCASM,i hope you know no news outlet from nigeria ...,pdp protest brainstorming session deji we nee...


Training model


In [100]:
sentences = []
labels = []
cont=[]

sentences.append(twitter_train['response'])
cont.append(twitter_train['context'])
labels.append(twitter_train['label'])

In [101]:
sentences

[0          i dont get this  obviously you do care or y...
 1         trying to protest about  talking about him a...
 2          he makes an insane about of money from the ...
 3         meanwhile trump wont even release his sat sc...
 4         pretty sure the antilincoln crowd claimed th...
                               ...                        
 4995     you dont  i have purchased a lot on amazon  c...
 4996     emotions you say  never knew that i think i  ...
 4997       you are so right   yes  silence is not priv...
 4998       another lazy delusional voter who takes the...
 4999      i hope you know no news outlet from nigeria ...
 Name: response, Length: 5000, dtype: object]

In [106]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
tokenizer.fit_on_texts(cont)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(sentences)
training_padded = pad_sequences(training_sequences, maxlen=100, padding='post', truncating='post')
train_cont_sequences=tokenizer.texts_to_sequences(cont)
train_cont_padded=pad_sequences(train_cont_sequences,100,padding='post',truncating='post')


AttributeError: ignored

In [ ]:
import numpy as np
training_padded = np.array(training_padded)
labels = np.array(labels)
train_cont_padded=np.array(train_cont_padded)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size=10000, embedding_dim, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

TEST

In [ ]:
twitter_test=pd.read_json('twitter_test.jsonl',lines=True)
twitter_test.head()

,context,response,id
0,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1
1,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2
2,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3
3,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4
4,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5


In [ ]:
twitter_test['response']=twitter_test['response'].str.replace('@USER', "") 
twitter_test['response']=twitter_test['response'].str.replace('\d+', '')
twitter_test['response']=twitter_test['response'].str.lower()
twitter_test['response']=twitter_test['response'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].apply(lambda x: ','.join(map(str, x)))
twitter_test['context']=twitter_test['context'].str.replace('@USER', "") 
twitter_test['context']=twitter_test['context'].str.lower()
twitter_test['context']=twitter_test['context'].str.replace('[^\w\s]','')
twitter_test['context']=twitter_test['context'].str.replace('\d+', '')

In [ ]:
twitter_test.head()

,context,response,id
0,well now that s problematic af url my year ...,my year old that just finished reading ni...,twitter_1
1,last week the fake news said that a section of...,how many verifiable lies has he told now d...,twitter_2
2,let s aplaud brett when he deserves it he co...,maybe docs just a scrub of a coach i mean ...,twitter_3
3,women generally hate this president whats up ...,is just a cover up for the real hate inside ...,twitter_4
4,dear media remoaners you excitedly sharing cl...,the irony being that he even has to ask why,twitter_5


In [ ]:
twitter_test.tail()

,context,response,id
1795,i have been a business customer of mweb for ...,is definitely the best out there no questi...,twitter_1796
1796,a woman refuses to have her temperature taken ...,ye let her out run wild and infect more chi...,twitter_1797
1797,the reason big government wants out is becaus...,thanks for that i would have never known,twitter_1798
1798,happy musicmonday and thanks for all your ...,yes also found this on new with loads of url a,twitter_1799
1799,not long wrapped on the amazing januarynd writ...,you still need to send the link to the fan ...,twitter_1800


In [ ]:
test_sentences=[]
test_cont=[]
test_labels=[]
test_sentences.append(twitter_test['response'])
test_cont.append(twitter_test['context'])
test_labels.append(twitter_test['label'])

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=100, padding='post', truncating='post')
testing_padded = np.array(testing_padded)
test_labels = np.array(test_labels)